## 1. Setup e Imports

In [ ]:
import sys
sys.path.append('../src')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('Set2')
%matplotlib inline

print('✓ Imports concluídos')

## 2. Carregamento das Métricas Gold

In [ ]:
GOLD_PATH = Path('../data/gold')

def load_gold_table(table_name):
    """Carrega tabela da camada Gold."""
    try:
        from deltalake import DeltaTable
        dt = DeltaTable(str(GOLD_PATH / table_name))
        df = dt.to_pandas()
        print(f"✓ {table_name}: {len(df):,} registros")
        return df
    except Exception as e:
        print(f"⚠ Erro ao carregar {table_name}: {e}")
        return None

# Carrega todas as tabelas Gold
print("Carregando métricas da camada Gold...\n")
df_velocidade = load_gold_table('velocidade_media_por_linha')
df_ativos = load_gold_table('onibus_ativos_por_periodo')
df_cobertura = load_gold_table('cobertura_geografica')
df_criticos = load_gold_table('pontos_criticos_velocidade')

## 3. Análise de Velocidade por Linha

In [ ]:
if df_velocidade is not None:
    print("=" * 60)
    print("VELOCIDADE MÉDIA POR LINHA")
    print("=" * 60)
    display(df_velocidade.head(10))
    
    # Top 10 linhas mais rápidas
    if 'numero_linha' in df_velocidade.columns:
        top_rapidas = df_velocidade.groupby('numero_linha')['velocidade_media'].mean().nlargest(10)
        
        plt.figure(figsize=(12, 6))
        top_rapidas.plot(kind='barh', color='green', edgecolor='black')
        plt.title('Top 10 Linhas Mais Rápidas', fontsize=14, fontweight='bold')
        plt.xlabel('Velocidade Média (km/h)')
        plt.ylabel('Linha')
        plt.tight_layout()
        plt.show()
    else:
        print("\n⚠ Coluna 'numero_linha' não encontrada")

In [ ]:
if df_velocidade is not None:
    # Distribuição de velocidades médias
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))
    
    axes[0].hist(df_velocidade['velocidade_media'], bins=30, 
                 color='skyblue', edgecolor='black')
    axes[0].set_title('Distribuição de Velocidades Médias', fontweight='bold')
    axes[0].set_xlabel('Velocidade Média (km/h)')
    axes[0].set_ylabel('Frequência')
    axes[0].axvline(df_velocidade['velocidade_media'].mean(), 
                    color='red', linestyle='--', 
                    label=f'Média: {df_velocidade["velocidade_media"].mean():.1f} km/h')
    axes[0].legend()
    
    axes[1].scatter(df_velocidade['velocidade_media'], 
                   df_velocidade['desvio_padrao'] if 'desvio_padrao' in df_velocidade.columns else None,
                   alpha=0.6, color='coral')
    axes[1].set_title('Velocidade Média vs Desvio Padrão', fontweight='bold')
    axes[1].set_xlabel('Velocidade Média (km/h)')
    axes[1].set_ylabel('Desvio Padrão')
    
    plt.tight_layout()
    plt.show()

## 4. Análise de Ônibus Ativos por Período

In [ ]:
if df_ativos is not None:
    print("=" * 60)
    print("ÔNIBUS ATIVOS POR PERÍODO")
    print("=" * 60)
    display(df_ativos.head())
    
    if 'hora' in df_ativos.columns:
        # Ônibus ativos por hora
        ativos_hora = df_ativos.groupby('hora')['total_registros'].sum().sort_index()
        
        plt.figure(figsize=(14, 6))
        ativos_hora.plot(kind='line', marker='o', linewidth=2, 
                        markersize=8, color='navy')
        plt.title('Atividade de Ônibus por Hora do Dia', 
                 fontsize=14, fontweight='bold')
        plt.xlabel('Hora')
        plt.ylabel('Total de Registros')
        plt.grid(alpha=0.3)
        plt.xticks(range(24))
        plt.tight_layout()
        plt.show()
        
        # Identifica horários de pico
        hora_pico = ativos_hora.idxmax()
        print(f"\n📊 Horário de Pico: {hora_pico}:00h ({ativos_hora.max():,} registros)")

In [ ]:
if df_ativos is not None and 'periodo_dia' in df_ativos.columns:
    # Distribuição por período do dia
    ativos_periodo = df_ativos.groupby('periodo_dia')['total_registros'].sum()
    
    fig, axes = plt.subplots(1, 2, figsize=(15, 6))
    
    # Pizza
    axes[0].pie(ativos_periodo, labels=ativos_periodo.index, autopct='%1.1f%%',
               startangle=90, colors=['#ff9999','#66b3ff','#99ff99','#ffcc99'])
    axes[0].set_title('Distribuição por Período do Dia', fontweight='bold')
    
    # Barras
    ativos_periodo.plot(kind='bar', ax=axes[1], color='teal', edgecolor='black')
    axes[1].set_title('Total de Registros por Período', fontweight='bold')
    axes[1].set_xlabel('Período')
    axes[1].set_ylabel('Total de Registros')
    axes[1].tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.show()

## 5. Análise de Cobertura Geográfica

In [ ]:
if df_cobertura is not None:
    print("=" * 60)
    print("COBERTURA GEOGRÁFICA")
    print("=" * 60)
    display(df_cobertura.head())
    
    if 'area_cobertura' in df_cobertura.columns:
        print(f"\n📍 Área Total de Cobertura: {df_cobertura['area_cobertura'].sum():.6f} graus²")
        print(f"📊 Total de Pontos Coletados: {df_cobertura['total_pontos'].sum():,}")
        
        # Visualização dos limites geográficos
        if all(col in df_cobertura.columns for col in ['lat_min', 'lat_max', 'lon_min', 'lon_max']):
            plt.figure(figsize=(10, 8))
            
            for idx, row in df_cobertura.iterrows():
                rect = plt.Rectangle(
                    (row['lon_min'], row['lat_min']),
                    row['lon_max'] - row['lon_min'],
                    row['lat_max'] - row['lat_min'],
                    fill=True, alpha=0.3, edgecolor='black'
                )
                plt.gca().add_patch(rect)
            
            plt.title('Áreas de Cobertura', fontsize=14, fontweight='bold')
            plt.xlabel('Longitude')
            plt.ylabel('Latitude')
            plt.grid(alpha=0.3)
            plt.tight_layout()
            plt.show()

## 6. Análise de Pontos Críticos

In [ ]:
if df_criticos is not None:
    print("=" * 60)
    print("PONTOS CRÍTICOS DE VELOCIDADE")
    print("=" * 60)
    print(f"\nTotal de pontos críticos identificados: {len(df_criticos):,}")
    
    display(df_criticos.head(10))
    
    if 'severidade' in df_criticos.columns:
        # Distribuição por severidade
        sev_counts = df_criticos['severidade'].value_counts()
        
        fig, axes = plt.subplots(1, 2, figsize=(15, 6))
        
        # Pizza
        colors = {'baixa': 'lightgreen', 'media': 'yellow', 
                 'alta': 'orange', 'critica': 'red'}
        sev_counts.plot(kind='pie', ax=axes[0], autopct='%1.1f%%',
                       colors=[colors.get(x, 'gray') for x in sev_counts.index])
        axes[0].set_title('Distribuição por Severidade', fontweight='bold')
        axes[0].set_ylabel('')
        
        # Barras
        sev_counts.plot(kind='bar', ax=axes[1], 
                       color=[colors.get(x, 'gray') for x in sev_counts.index],
                       edgecolor='black')
        axes[1].set_title('Contagem por Severidade', fontweight='bold')
        axes[1].set_xlabel('Severidade')
        axes[1].set_ylabel('Quantidade')
        axes[1].tick_params(axis='x', rotation=45)
        
        plt.tight_layout()
        plt.show()

In [ ]:
if df_criticos is not None and all(col in df_criticos.columns for col in ['lat_grid', 'lon_grid', 'ocorrencias']):
    # Mapa de calor de pontos críticos
    plt.figure(figsize=(12, 10))
    
    scatter = plt.scatter(
        df_criticos['lon_grid'],
        df_criticos['lat_grid'],
        s=df_criticos['ocorrencias'] * 10,
        c=df_criticos['velocidade_media'],
        cmap='RdYlGn_r',
        alpha=0.6,
        edgecolors='black',
        linewidth=0.5
    )
    
    plt.colorbar(scatter, label='Velocidade Média (km/h)')
    plt.title('Mapa de Pontos Críticos de Velocidade\n(tamanho = ocorrências)', 
             fontsize=14, fontweight='bold')
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')
    plt.grid(alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    # Top 10 pontos mais críticos
    print("\n🚨 Top 10 Pontos Mais Críticos:")
    top_criticos = df_criticos.nlargest(10, 'ocorrencias')[['lat_grid', 'lon_grid', 'ocorrencias', 'velocidade_media']]
    display(top_criticos)

## 7. Resumo Executivo

In [ ]:
print("=" * 60)
print("RESUMO EXECUTIVO - MOBILIDADE URBANA BH")
print("=" * 60)

if df_velocidade is not None:
    print(f"\n📊 VELOCIDADE:")
    print(f"   • Velocidade média geral: {df_velocidade['velocidade_media'].mean():.1f} km/h")
    print(f"   • Variação: {df_velocidade['velocidade_media'].min():.1f} - {df_velocidade['velocidade_media'].max():.1f} km/h")

if df_ativos is not None:
    print(f"\n🚍 FROTA ATIVA:")
    print(f"   • Total de registros: {df_ativos['total_registros'].sum():,}")
    if 'total_onibus_unicos' in df_ativos.columns:
        total_unicos = df_ativos['total_onibus_unicos'].sum()
        if total_unicos > 0:
            print(f"   • Ônibus únicos: {total_unicos:,}")

if df_cobertura is not None:
    print(f"\n📍 COBERTURA:")
    print(f"   • Pontos coletados: {df_cobertura['total_pontos'].sum():,}")
    print(f"   • Área de cobertura: {df_cobertura['area_cobertura'].sum():.6f} graus²")

if df_criticos is not None:
    print(f"\n⚠️  PONTOS CRÍTICOS:")
    print(f"   • Total identificados: {len(df_criticos):,}")
    if 'severidade' in df_criticos.columns:
        criticos = (df_criticos['severidade'] == 'critica').sum()
        print(f"   • Críticos: {criticos}")

print("\n" + "=" * 60)

## 8. Conclusões

Este notebook apresentou análises das principais métricas de negócio:

- ✅ **Velocidade**: Identificamos padrões e linhas mais/menos eficientes
- ✅ **Atividade**: Analisamos horários de pico e períodos de maior movimento
- ✅ **Cobertura**: Mapeamos a abrangência geográfica do sistema
- ✅ **Gargalos**: Identificamos pontos críticos que necessitam atenção

### Próximos Passos

1. Análise temporal mais profunda (tendências, sazonalidade)
2. Correlação com eventos externos (clima, eventos)
3. Modelos preditivos de velocidade e demanda
4. Otimização de rotas baseada em dados históricos